In [3]:
import pandas as pd
import numpy as np
import re

import gensim
from gensim import corpora,models
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2018)

#!pip install pyLDAvis

import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

#import warnings
#warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
# this will slove broken link issue with nltk using ssl.
#import ssl
#try:
#    _create_unverified_https_context = ssl._create_unverified_context
#except AttributeError:
#    pass
#else:
#    ssl._create_default_https_context = _create_unverified_https_context

In [4]:
import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
#from google.colab import drive
#drive.mount('/content/drive')

In [7]:
data = pd.read_csv('/content/drive/My Drive/result1.csv');

In [8]:
data.shape

(162, 8)

In [16]:
data.dropna(subset=['content'], inplace = True)
data.dropna(subset = ['topic'], inplace = True)

In [17]:
data.shape

(148, 8)

In [9]:
# data preprocessing
stemmer = SnowballStemmer(language='english',ignore_stopwords=True)
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [10]:
# preview after preprocessing
doc_sample = data.content[0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['["Prime', 'Minister', '(PM)', 'Narendra', 'Modi', 'recently', 'announced', 'that', 'India', 'must', 'become', '","(self-reliant).', 'One', 'aspect', 'of', 'this', 'could', 'be', 'that', 'India', 'will', 'remove', 'barriers', 'within', 'its', 'internal', 'markets', 'to', 'truly', 'become', 'a', 'single', 'market.', 'It', 'will', 'remove', 'the', 'hurdles', 'to', 'efficiency', 'improvements', 'and', 'become', 'more', 'competitive.', 'The', 'Goods', 'and', 'Services', 'Tax', '(GST)', 'was', 'a', 'step', 'in', 'this', 'direction.', 'Recent', 'decisions', 'to', 'remove', 'hurdles', 'in', 'inter-state', 'agricultural', 'trade', 'are', 'also', 'similar.', 'For', 'agricultural', 'and', 'industrial', 'products,', 'as', 'well', 'as', 'capital,', 'India', 'is', 'increasingly', 'becoming', 'a', 'single', 'market.', 'The', 'creation', 'of', 'a', 'barrier-free', 'domestic', 'market', 'is', 'also', 'an', 'intent', 'reflected', 'in', 'Article', '301', 'of', 'the', 'Constitution."

In [18]:
# Preprocess the headline text, saving the results as ‘processed_docs’
processed_docs = data['content'].map(preprocess)
#processed_docs.head(5)
data['processed_heading'] = processed_docs

In [22]:
#data['removed_list'] = data['processed_heading'].apply(lambda x: ' '.join(x))
#data.head(2)
#removed_list = data.removed_list

In [24]:
#Create a dictionary from ‘processed_docs’ containing the number of times a word appears in the training set
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 access
1 add
2 advantag
3 agricultur
4 allow
5 announc
6 anti
7 appli
8 approv
9 articl
10 aspect


In [ ]:
#dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [27]:
# gensim doc2bow
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[0]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 2),
 (4, 1),
 (5, 3),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 3),
 (10, 1),
 (11, 1),
 (12, 1),
 (13, 1),
 (14, 1),
 (15, 2),
 (16, 1),
 (17, 1),
 (18, 1),
 (19, 3),
 (20, 3),
 (21, 1),
 (22, 2),
 (23, 1),
 (24, 2),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 2),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 3),
 (39, 1),
 (40, 1),
 (41, 1),
 (42, 1),
 (43, 2),
 (44, 2),
 (45, 2),
 (46, 1),
 (47, 1),
 (48, 1),
 (49, 1),
 (50, 1),
 (51, 1),
 (52, 1),
 (53, 1),
 (54, 3),
 (55, 3),
 (56, 1),
 (57, 1),
 (58, 1),
 (59, 1),
 (60, 3),
 (61, 1),
 (62, 1),
 (63, 1),
 (64, 2),
 (65, 1),
 (66, 1),
 (67, 1),
 (68, 2),
 (69, 4),
 (70, 3),
 (71, 1),
 (72, 1),
 (73, 2),
 (74, 2),
 (75, 1),
 (76, 2),
 (77, 1),
 (78, 2),
 (79, 1),
 (80, 1),
 (81, 1),
 (82, 1),
 (83, 1),
 (84, 1),
 (85, 1),
 (86, 3),
 (87, 1),
 (88, 1),
 (89, 1),
 (90, 1),
 (91, 4),
 (92, 1),
 (93, 1),
 (94, 1),
 (95, 2),
 (96, 1),
 (97, 2),
 (98, 1),
 (99, 2),
 (100, 2),

In [28]:
# Preview Bag Of Words for our sample preprocessed document.
bow_doc_0 = bow_corpus[0]
for i in range(len(bow_doc_0)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_0[i][0], 
                                               dictionary[bow_doc_0[i][0]], 
bow_doc_0[i][1]))

Word 0 ("access") appears 1 time.
Word 1 ("add") appears 1 time.
Word 2 ("advantag") appears 1 time.
Word 3 ("agricultur") appears 2 time.
Word 4 ("allow") appears 1 time.
Word 5 ("announc") appears 3 time.
Word 6 ("anti") appears 1 time.
Word 7 ("appli") appears 1 time.
Word 8 ("approv") appears 1 time.
Word 9 ("articl") appears 3 time.
Word 10 ("aspect") appears 1 time.
Word 11 ("auto") appears 1 time.
Word 12 ("avail") appears 1 time.
Word 13 ("backdrop") appears 1 time.
Word 14 ("bangalor") appears 1 time.
Word 15 ("barrier") appears 2 time.
Word 16 ("base") appears 1 time.
Word 17 ("benefici") appears 1 time.
Word 18 ("beneficiari") appears 1 time.
Word 19 ("benefit") appears 3 time.
Word 20 ("bengaluru") appears 3 time.
Word 21 ("best") appears 1 time.
Word 22 ("bihar") appears 2 time.
Word 23 ("brave") appears 1 time.
Word 24 ("capit") appears 2 time.
Word 25 ("capita") appears 1 time.
Word 26 ("card") appears 1 time.
Word 27 ("casual") appears 1 time.
Word 28 ("categori") appea

In [29]:
# tf-idf
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
for doc in corpus_tfidf:
    print(doc)
    break

[(0, 0.023482283267781765), (1, 0.009404684218986006), (2, 0.02579755493806464), (3, 0.08043534940659296), (4, 0.013692706843115665), (5, 0.046022690392083315), (6, 0.023482283267781765), (7, 0.031376018749902874), (8, 0.02455694686361218), (9, 0.09412805624970863), (10, 0.02906074707962001), (11, 0.03463921089145825), (12, 0.01652076479423945), (13, 0.02906074707962001), (14, 0.04021767470329648), (15, 0.0692784217829165), (16, 0.014640627314388163), (17, 0.02579755493806464), (18, 0.04021767470329648), (19, 0.0676030883895278), (20, 0.12065302410988943), (21, 0.016107955228600358), (22, 0.05812149415924002), (23, 0.03463921089145825), (24, 0.03684645379776645), (25, 0.04021767470329648), (26, 0.02579755493806464), (27, 0.03463921089145825), (28, 0.02906074707962001), (29, 0.031376018749902874), (30, 0.019574905647757514), (31, 0.017415911161147962), (32, 0.04696456653556353), (33, 0.04021767470329648), (34, 0.02579755493806464), (35, 0.02906074707962001), (36, 0.021686419040438592), 

In [ ]:
#LDA using Bag of Words
# # Build LDA model
# lda_model = gensim.models.ldamodel.LdaModel(corpus=bow_corpus,
#                                            id2word=dictionary,
#                                            num_topics=10, 
#                                            random_state=100,
#                                            update_every=1,
#                                            chunksize=100,
#                                            passes=10,
#                                            alpha='auto',
#                                            per_word_topics=True)

In [31]:
# Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [32]:
# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.014*"india" + 0.013*"china" + 0.007*"say" + 0.007*"labour" + 0.006*"indian" + 0.005*"militari" + 0.005*"chines" + 0.004*"power" + 0.004*"nation" + 0.004*"state"
Topic: 1 
Words: 0.009*"say" + 0.008*"test" + 0.007*"patient" + 0.007*"covid" + 0.006*"india" + 0.006*"friend" + 0.005*"today" + 0.005*"exercis" + 0.005*"lucki" + 0.005*"case"
Topic: 2 
Words: 0.012*"result" + 0.008*"board" + 0.007*"share" + 0.007*"say" + 0.007*"class" + 0.007*"time" + 0.006*"post" + 0.004*"year" + 0.004*"dubey" + 0.004*"actor"
Topic: 3 
Words: 0.020*"say" + 0.007*"delhi" + 0.006*"learn" + 0.006*"govern" + 0.006*"school" + 0.005*"covid" + 0.004*"test" + 0.004*"class" + 0.004*"teacher" + 0.004*"polic"
Topic: 4 
Words: 0.011*"say" + 0.011*"india" + 0.006*"china" + 0.005*"actor" + 0.005*"sushant" + 0.005*"time" + 0.005*"year" + 0.004*"post" + 0.004*"like" + 0.004*"peopl"
Topic: 5 
Words: 0.006*"say" + 0.006*"time" + 0.006*"peopl" + 0.005*"year" + 0.005*"india" + 0.005*"line" + 0.004*"test" + 0.0

In [33]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -8.24305717280266

Coherence Score:  0.33510514984144024


In [34]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.000288  0.042000       1        1  12.812829
3      0.014162 -0.093457       2        1  12.018830
6      0.042771  0.006993       3        1  11.952091
8      0.062473 -0.002646       4        1  10.705054
4     -0.051246  0.054833       5        1   9.824071
2     -0.086967 -0.046753       6        1   9.624974
1      0.046827  0.051422       7        1   9.486127
7      0.088651 -0.028521       8        1   8.348923
5     -0.038607  0.033721       9        1   8.032031
9     -0.077776 -0.017592      10        1   7.195073, topic_info=        Term        Freq       Total Category  logprob  loglift
614     test  104.000000  104.000000  Default  30.0000  30.0000
274    china  116.000000  116.000000  Default  29.0000  29.0000
849   result   64.000000   64.000000  Default  28.0000  28.0000
1500    love   69.000000   69.000000  Default  27.0000  27.0000
1513   share   79.000000   79.000000  Default  26.0000  26.0000
...      ...         ...         ...      ...      ...      ...
563   person    5.726450   53.240566  Topic10  -5.9566   0.4020
1151    know    5.483520   38.363319  Topic10  -5.9999   0.6864
319       go    5.566634   52.626358  Topic10  -5.9849   0.3853
224    state    5.609883  121.127632  Topic10  -5.9771  -0.4405
113    india    5.593893  215.949860  Topic10  -5.9800  -1.0216

[728 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
5160      7  0.697302  actigraph
1102      1  0.063496      actor
1102      2  0.142866      actor
1102      3  0.031748      actor
1102      4  0.031748      actor
...     ...       ...        ...
1450      3  0.750154       zhao
4875      4  0.083789     zodiac
4875      7  0.754100     zodiac
4875      8  0.083789     zodiac
4923      3  0.834645      zonal

[2169 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 4, 7, 9, 5, 3, 2, 8, 6, 10])

In [ ]:
#Gensim provides a wrapper to implement Mallet’s LDA from within Gensim itself. 
#You only need to download the zipfile, unzip it and provide the path to mallet in the unzipped directory to gensim.models.wrappers.LdaMallet
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
! wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

In [ ]:
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=bow_corpus, num_topics=10, id2word=dictionary)

In [ ]:
# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in ldamallet.show_topics(formatted=False):
    print('Topic: {} \nWords: {}'.format(idx, topic))
print()

In [ ]:
# Compute Perplexity
# print('\nPerplexity: ', ldamallet.log_perplexity(bow_corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [ ]:
# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(ldamallet, bow_corpus, dictionary)
# vis

In [36]:
# Train our lda model using gensim.models.LdaMulticore and save it to ‘lda_model’
lda_model_tf_idf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [37]:
# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in lda_model_tf_idf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.001*"coal" + 0.001*"maya" + 0.001*"saroj" + 0.001*"kravitz" + 0.001*"athlet" + 0.001*"coach" + 0.001*"rial" + 0.001*"bless" + 0.001*"exercis" + 0.001*"sleep"
Topic: 1 
Words: 0.001*"china" + 0.001*"chines" + 0.001*"india" + 0.001*"ganguli" + 0.001*"militari" + 0.001*"border" + 0.001*"crpf" + 0.001*"love" + 0.001*"foreign" + 0.001*"share"
Topic: 2 
Words: 0.001*"board" + 0.001*"result" + 0.001*"shah" + 0.001*"exam" + 0.001*"refund" + 0.001*"mpbse" + 0.001*"rain" + 0.001*"china" + 0.001*"celina" + 0.001*"film"
Topic: 3 
Words: 0.001*"singh" + 0.001*"intel" + 0.001*"assam" + 0.001*"video" + 0.001*"lola" + 0.001*"deepika" + 0.001*"woman" + 0.001*"morata" + 0.001*"oximet" + 0.001*"billion"
Topic: 4 
Words: 0.001*"dubey" + 0.001*"test" + 0.001*"prasar" + 0.001*"bharati" + 0.001*"navya" + 0.001*"gemma" + 0.001*"throne" + 0.001*"class" + 0.001*"case" + 0.001*"app"
Topic: 5 
Words: 0.002*"school" + 0.002*"learn" + 0.001*"india" + 0.001*"china" + 0.001*"delhi" + 0.001*"sushant

In [39]:
# Compute Perplexity
print('\nPerplexity: ', lda_model_tf_idf.log_perplexity(corpus_tfidf))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model_tf_idf, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for


Perplexity:  -13.64994474715381

Coherence Score:  0.5792533155231394


In [40]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model_tf_idf, corpus_tfidf, dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.023490 -0.002744       1        1  18.270704
7      0.001718  0.020211       2        1  12.853691
9     -0.000333 -0.007589       3        1  12.725726
1     -0.000992 -0.007941       4        1  12.033597
2     -0.006427 -0.000536       5        1   9.875732
3     -0.005494 -0.002243       6        1   9.204123
4     -0.000341  0.001649       7        1   7.941736
0     -0.003907 -0.000223       8        1   7.130217
6     -0.003523  0.001190       9        1   5.687336
8     -0.004192 -0.001774      10        1   4.277136, topic_info=          Term      Freq     Total Category  logprob  loglift
4519     dubey  0.000000  0.000000  Default  30.0000  30.0000
5740  tallulah  0.000000  0.000000  Default  29.0000  29.0000
2533      exam  0.000000  0.000000  Default  28.0000  28.0000
4099      coal  0.000000  0.000000  Default  27.0000  27.0000
1581     board  0.000000  0.000000  Default  26.0000  26.0000
...        ...       ...       ...      ...      ...      ...
164      point  0.029013  0.354922  Topic10  -7.5206   0.6477
1851    season  0.027254  0.324729  Topic10  -7.5831   0.6741
1239  district  0.024654  0.311312  Topic10  -7.6834   0.6160
700   signific  0.023989  0.281391  Topic10  -7.7108   0.6897
941       draw  0.022837  0.249668  Topic10  -7.7600   0.7601

[467 rows x 6 columns], token_table=Empty DataFrame
Columns: [Topic, Freq, Term]
Index: [], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 10, 2, 3, 4, 5, 1, 7, 9])

In [ ]:
mallet_path = 'mallet-2.0.8/bin/mallet' # update this path
ldamallet_tf_idf = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus_tfidf, num_topics=10, id2word=dictionary)

In [ ]:
# For each topic, we will explore the words occuring in that topic and its relative weight.
for idx, topic in ldamallet_tf_idf.show_topics(formatted=False):
    print('Topic: {} \nWords: {}'.format(idx, topic))
print()

In [ ]:
# Compute Perplexity
#print('\nPerplexity: ', lda_model_tf_idf.log_perplexity(corpus_tfidf))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamallet_tf_idf, texts=processed_docs, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
# Visualize the topics
# pyLDAvis.enable_notebook()
# vis = pyLDAvis.gensim.prepare(ldamallet_tf_idf, corpus_tfidf, dictionary)
# vis

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus, texts=processed_docs, start=2, limit=40, step=6)

In [ ]:
# Show graph
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
# Select the model and print the topics
optimal_model = model_list[2]
model_topics = optimal_model.show_topics(formatted=False)
print(optimal_model.print_topics(num_words=10))

In [ ]:
#finding the dominant topic in each sentence
# de-tokenization
detokenized_doc = []
for text in processed_docs:
    t = ' '.join(text)
    detokenized_doc.append(t)

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=bow_corpus, texts=detokenized_doc):

    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=bow_corpus, texts=detokenized_doc)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
#Find the most representative document for each topic
# Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head()

In [ ]:
#Topic distribution across documents
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

In [ ]:
# Latent Sentiment Analysis
# Document Term Matrix
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000, # keep top 1000 terms 
max_df = 0.5, 
smooth_idf=True)

X = vectorizer.fit_transform(detokenized_doc)

X.shape # check shape of the document-term matrix

In [ ]:
from sklearn.decomposition import TruncatedSVD

# SVD represent documents and terms in vectors 
svd_model = TruncatedSVD(n_components=20, algorithm='randomized', n_iter=100, random_state=122)

svd_model.fit(X)

len(svd_model.components_)

In [ ]:
#The components of svd_model are our topics, and we can access them using svdmodel.components. 
#Finally, let’s print a few most important words in each of the 20 topics and see how our model has done

In [ ]:
terms = vectorizer.get_feature_names()

for i, comp in enumerate(svd_model.components_):
    terms_comp = zip(terms, comp)
    sorted_terms = sorted(terms_comp, key= lambda x:x[1], reverse=True)[:7]
    print("Topic "+str(i)+": ")
    for t in sorted_terms:
        print(t[0])
        print(" ")

In [ ]:
#Topics Visualization
import umap.umap_ as umap

X_topics = svd_model.fit_transform(X)
embedding = umap.UMAP(n_neighbors=150, min_dist=0.5, random_state=12).fit_transform(X_topics)

plt.figure(figsize=(7,5))
plt.scatter(embedding[:, 0], embedding[:, 1], 
c = dataset.target,
s = 10, # size
edgecolor='none'
)
plt.show()